In [2]:
# Setting Test Automation Parameters
CLUSTER_LOCATION = "LR1"
HTTP_PROXY = "http://hpeproxy.its.hpecorp.net:80"

if CLUSTER_LOCATION in ('LR1','AIRGAP'):
    !pip --proxy {HTTP_PROXY} install tblib
else:
    !pip install tblib

  Obtaining dependency information for tblib from https://files.pythonhosted.org/packages/9e/bd/ccb241b97e39dd8ec143418f89b3fd5752f872c862877a0f1b2d9fb9e815/tblib-2.0.0-py3-none-any.whl.metadata
  Using cached tblib-2.0.0-py3-none-any.whl.metadata (25 kB)
Using cached tblib-2.0.0-py3-none-any.whl (11 kB)


In [1]:
import ray
import numpy as np
import tensorflow as tf

# Please make sure about below points
# 1. If you execute the job from a notebook, then your connection URL should be as below
#    ray.init(address="ray://kuberay-head-svc.kuberay:10001")
# 2. For this example tensorflow package needs to installed, either install it manually or pass the installation command in init command as below
#    ray.init(address="ray://kuberay-head-svc.kuberay:10001", runtime_env={"pip": ["tensorflow"]})
# 3. If you have proxy, then you need to add following dictionary into runtime_env parameter in ray.init method
#    "env_vars": {"http_proxy": "<PROXY_ADDRESS>", "https_proxy": "<PROXY_ADDRESS>"}
# ray.init(address="ray://kuberay-head-svc.kuberay:10001", runtime_env={"pip": ["tensorflow"]})

ray.init(address="ray://kuberay-head-svc.kuberay:10001", ignore_reinit_error=True)

@ray.remote(num_gpus=1)
def gpu_operation(data):
    # Perform GPU-accelerated operation
    # Example: Square each element of the input array using TensorFlow
    with tf.device("/GPU:0"):
        tensor = tf.constant(data)
        squared = tf.square(tensor)
        result = squared.numpy()
    return result

# data
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
num_chunks = 2

# Split the data into chunks
data_chunks = np.array_split(data, num_chunks)

# Submit GPU operations for execution in parallel
result_refs = [gpu_operation.remote(chunk) for chunk in data_chunks]

# Retrieve the results
results = ray.get(result_refs)

print(results)

ray.shutdown()

2023-07-30 19:35:05.508868: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-30 19:35:05.510718: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-30 19:35:05.550742: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-30 19:35:05.551404: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 19:35:06.260530: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

[array([ 1,  4,  9, 16, 25]), array([ 36,  49,  64,  81, 100])]


(gpu_operation pid=6765, ip=10.224.0.185) 2023-07-30 12:35:20.034089: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7994 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB MIG 2g.10gb, pci bus id: 0000:86:00.0, compute capability: 8.0
